In [1]:
# from sklearn.datasets import make_classification
# from sklearn.cross_validation import StratifiedShuffleSplit
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import numpy as np

In [ ]:
# We use a utility to generate artificial classification data.
X, y = make_classification(n_samples=100, n_informative=10, n_classes=3)
sss = StratifiedShuffleSplit(y, n_iter=1, test_size=0.5, random_state=0)
for train_idx, test_idx in sss:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))    

In [30]:
def calculate_AER(S, P, A):
    S, P, A = np.array(S), np.array(P), np.array(A)
    s_a, p_a, len_s, len_a = 0, 0, 0, 0
    for s, p, a in zip(S, P, A):
        s_a += len(list(set(s).intersection(a)))
        p_a += len(list(set(p).intersection(a)))
        len_s += len(s[s != ""])
        len_a += len(a[a != ""])
    print ("s_a", s_a)
    p_a += s_a
    print ("p_a", p_a)
    aer = (s_a + p_a) / (len_s + len_a)
    print ("aer", 1.-aer)
    
    return 1. - aer 


def calculate_one_AER(S, P, A):
    S, P, A = np.array(S), np.array(P), np.array(A)
    s_a = len(list(set(S).intersection(A)))
    print ("s_a", s_a)
    p_a = len(list(set(P).intersection(A))) + s_a
    print ("p_a", p_a)
    aer = (s_a + p_a) / (len(S[S != ""]) + len(A[A != ""]))
    print ("aer", 1.-aer)
    
    return 1. - aer 
    
def write_file(strs, file_name):
    alignment_test = open(file_name,"w", encoding='utf8') 
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

In [31]:
S = [['3-2', '1-1']]
P = [['2-2']]
A = [['1-2', '1-1', '3-2']]
calculate_AER(S, P, A)

s_a 2
p_a 2
aer 0.19999999999999996


0.19999999999999996

In [17]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open

docXml = './celex_21990A0714.m.wa'
root = minidom.parse(docXml).getElementsByTagName("sentences")

english_sentences = []
czech_sentences = []
sure = []
possible = []

for sent in root[0].childNodes :
    
    #on retire les noeuds vides
    if sent.localName == None : continue

    sentID = sent.getAttribute('id')

    #pour chaque noeud de phrase on parcourt les tokens de la phrase, on regarde leur longueur et on calcule
    #le numero du premier charactère ... sera utile pour la suite pour avoir l'ID html du charactère correspondant
    tokenNodes = sent.childNodes
    
    english_sentences.append(tokenNodes[1].childNodes[0].data)
    czech_sentences.append(tokenNodes[3].childNodes[0].data)
    sure.append(tokenNodes[5].childNodes[0].data)
    try:
        possible.append(tokenNodes[7].childNodes[0].data)
    except Exception: 
        possible.append('')

        
write_file(english_sentences, "alignment_test.en")
write_file(czech_sentences, "alignment_test.cz")

In [19]:
result = open("corpus.en2cz.gdfa.ali", "r")
aer = 0.
S, P, A = [], [], []

for line, s_, p_ in zip(result, sure, possible):
    a = line.strip().split(" ")
    for number, i in zip(a, range(len(a))):
        n1 = int(number.split("-")[0]) + 1 
        n2 = int(number.split("-")[1]) + 1 
        number = str(n1) + "-" + str(n2)
        a[i] = number
    S.append(s_.strip().split(" "))
    P.append(p_.strip().split(" "))
    A.append(a)
    
aer = calculate_AER(S, P, A)

print(aer)
result.close()

['1-1', '3-2', '3-3', '3-4', '3-5', '4-6', '5-7', '5-8', '5-9', '5-10', '8-12'] ['2-3', '6-11', '7-12'] ['1-1', '2-2', '3-3', '4-4', '5-5', '6-6', '7-7', '8-8']
['2-1', '5-4', '7-2', '8-3', '9-5', '10-5', '11-6', '12-7', '14-8', '14-9', '14-10', '14-11', '15-15', '19-18', '21-19', '21-20', '21-21', '21-22', '21-23', '21-24', '22-25', '25-26', '26-27', '28-28', '29-29', '30-30', '31-31', '32-32', '33-33', '34-34', '35-35', '37-36', '39-39', '40-37', '41-40'] ['1-1', '3-4', '4-4', '6-4', '13-9', '15-12', '15-13', '15-14', '16-16', '16-17', '18-18', '20-20', '24-28', '36-36', '38-37', '39-38'] ['1-1', '2-2', '3-3', '4-4', '5-5', '6-6', '7-7', '8-8', '9-9', '10-10', '11-11', '12-12', '13-13', '14-14', '15-15', '16-16', '17-17', '18-18', '19-19', '20-20', '21-21', '22-22', '23-23', '24-24', '25-25', '26-26', '27-27', '27-29', '28-28', '29-27', '30-32', '31-32', '32-30', '32-31', '32-32', '33-33', '34-34', '35-35', '36-36', '37-37', '38-38', '39-39', '40-40', '41-41']
['2-3', '3-1', '4-4', '

# Preprocessing training file: En-Cz

In [7]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open
import glob

from lxml import html
from lxml import etree
# tree = html.parse(path)

def write_file(strs, file_name):
    alignment_test = open(file_name,"w", encoding='utf8') 
    print("Writing file: ", file_name)
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

english_sentences = []
czech_sentences = []
sure = []
possible = []

file = "E:/Working/Intership2017/en-cz/raw/training/news-commentary-v11.cs-en.xliff"
print("Processing file:", file)
parser = html.parse(file)
sentences = parser.getroot().getchildren()[0].getchildren()[0].getchildren()[0].getchildren()
for s in sentences:
    child = s.getchildren()
    values = []
    for c in child:
        if (c.text != None):
            values.append(str(c.text))
        else:
            values.append('')
    english_sentences.append(values[1])
    czech_sentences.append(values[0])

assert len(english_sentences)==len(czech_sentences)

out_dir = "/people/vu/Projects/2017-NeuralAlignments/data/en-cz/formatted/testing/"
write_file(english_sentences, out_dir + "training.en-cz.en")
write_file(czech_sentences, out_dir + "training.en-cz.cz")
print("Done !")

Processing file: E:/Working/Intership2017/en-cz/raw/training/news-commentary-v11.cs-en.xliff
Writing file:  training.en-cz.en
Writing file:  training.en-cz.cz
Done !


# Preprocessing testing file: En-Cz

In [4]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open
import glob

from lxml import html
from lxml import etree
# tree = html.parse(path)

def write_file(strs, file_name):
    alignment_test = open(file_name, "w", encoding='utf8') 
    print("Writing file: ", file_name)
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

english_sentences = []
czech_sentences = []
sure = []
possible = []

files = glob.glob("/people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/*.wa")
for file in files:
    print("Processing file:", file)
    parser = html.parse(file)
    root = parser.getiterator("sentences")
    for r in root:
        sentences = r.getchildren()
        for s in sentences:
            child = s.getchildren()
            values = []
            for c in child:
                if (c.text != None):
                    values.append(str(c.text))
                else:
                    values.append('')
            english_sentences.append(values[0])
            czech_sentences.append(values[1])
            sure.append(values[2])
            possible.append(values[3])
    
    assert len(english_sentences)==len(czech_sentences)==len(sure)==len(possible)

out_dir = "/people/vu/Projects/2017-NeuralAlignments/data/en-cz/formatted/testing/"
write_file(english_sentences, out_dir + "testing.en-cz.en")
write_file(czech_sentences, out_dir + "testing.en-cz.cz")
align_test = []
for (s, p) in zip(sure, possible):
    align_test.append(s)
    align_test.append(p)
write_file(align_test, out_dir + "testing.en-cz.aligment")
# Fast-align format
fast_align_test = []
for (e, z) in zip(english_sentences, czech_sentences):
    fast_align_test.append(e + " ||| " + z)
write_file(fast_align_test, out_dir + "testing.en-cz.fast-align")
    
print("Done !")

Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/books_three_men_in_boat.wa
Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/celex_21990A0714.wa
Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/celex_22000A0411.wa
Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/celex_31958L0003.wa
Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/celex_31962L2645.wa
Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/celex_31965L0079.wa
Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/celex_31970L0220.wa
Processing file: /people/vu/Projects/2017-NeuralAlignments/data/en-cz/raw/testing/aligned_cz-en_1.0/celex_31975L0033.wa
Processing file: /people/vu/Proje

# Fast-Align En-cz evolution

In [55]:
result_file = open("/people/vu/Projects/2017-NeuralAlignments/exp-bach/en-cz/fast_align/result.en-cz.align")
target_file = open("/people/vu/Projects/2017-NeuralAlignments/data/en-cz/formatted/testing/testing.en-cz.aligment")


# Index starts from 0
result_lines = result_file.readlines()
result_lines = [str(line[:-1]) for line in result_lines]
# Index starts from 1
target_lines = target_file.readlines()
target_lines = [str(line[:-1]) for line in target_lines]
target_lines = np.reshape(target_lines, (2501, 2))

sure = target_lines[:,0]
possible = target_lines[:,1]
result = result_lines

S, P, A = [], [], []

# Split and plus 1 to result indexes
for line, s_, p_ in zip(result, sure, possible):
    a = line.strip().split(" ")
    for i, number in enumerate(a):
        n1 = int(number.split("-")[0]) + 1 
        n2 = int(number.split("-")[1]) + 1 
        number = str(n1) + "-" + str(n2)
        a[i] = number
    S.append(s_.strip().split(" "))
    P.append(p_.strip().split(" "))
    A.append(a)

calculate_AER(S, P, A)

s_a 30184
p_a 34917
aer 0.3732635044718069


0.3732635044718069

# GIZA++

In [172]:
import re

def write_file(strs, file_name):
    alignment_test = open(file_name, "w", encoding='utf8') 
    print("Writing file: ", file_name)
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()


giza_file = open("/people/vu/Projects/2017-NeuralAlignments/exp-bach/en-cz/GIZA++/v1.A3.final", encoding='utf8')

giza_result = giza_file.readlines()
print(np.shape(giza_result))
giza_result = np.reshape(giza_result, (np.shape(giza_result)[0]/3, 3))[:,2]
giza_result = [re.findall(r"\(\{([^\(]+)\}\)", s[:-1], re.I) for s in giza_result]

out_result = []
for row in giza_result:
    align = ""
    for i, value in enumerate(row):
        if i == 0:
            continue
        align_sublist = value.split()
        for a in align_sublist:
            align += str(i) + "-" + str(a) + " "
    out_result.append(str.strip(align))
    
write_file(out_result, "/people/vu/Projects/2017-NeuralAlignments/exp-bach/en-cz/GIZA++/result.en-cz.align")
print("Done !")

(7503,)


/people/vu/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  result = getattr(asarray(obj), method)(*args, **kwds)


Writing file:  /people/vu/Projects/2017-NeuralAlignments/exp-bach/en-cz/GIZA++/result.en-cz.align


# GIZA++ evolution

In [173]:
result_file = open("/people/vu/Projects/2017-NeuralAlignments/exp-bach/en-cz/GIZA++/result.en-cz.align")
target_file = open("/people/vu/Projects/2017-NeuralAlignments/data/en-cz/formatted/testing/testing.en-cz.aligment")

# Index starts from 1
result_lines = result_file.readlines()
result_lines = [str(line[:-1]) for line in result_lines]
# Index starts from 1
target_lines = target_file.readlines()
target_lines = [str(line[:-1]) for line in target_lines]
target_lines = np.reshape(target_lines, (2501, 2))

sure = target_lines[:,0]
possible = target_lines[:,1]
result = result_lines

S, P, A = [], [], []

# Split and plus 1 to result indexes
for a_, s_, p_ in zip(result, sure, possible):
    S.append(s_.strip().split(" "))
    P.append(p_.strip().split(" "))
    A.append(a_.strip().split(" "))

calculate_AER(S, P, A)

s_a 23883
p_a 25799
aer 0.45722307801557904


0.45722307801557904

# Preprocessing training file: En-Fr

In [12]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open
import glob
import os

from lxml import html
from lxml import etree

def write_file(strs, file_name):
    alignment_test = open(file_name, "w", encoding='utf8') 
    print("Writing file: ", file_name)
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

def write_file_list(file_list, file_name):
    with open(file_name, "w", encoding='utf8') as outfile:
        for fname in file_list:
            print("Processing file:", os.path.basename(fname))
            with open(fname, encoding='utf8') as infile:
                outfile.write(infile.read())
                outfile.write("\n")

out_dir = "/people/vu/Projects/2017-NeuralAlignments/data/en-fr/formatted/training/"
e_house_files = glob.glob("/people/vu/Projects/2017-NeuralAlignments/data/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/*.e")
f_house_files = glob.glob("/people/vu/Projects/2017-NeuralAlignments/data/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/*.f")
write_file_list(e_house_files, out_dir + "training.house.en-fr.en")
write_file_list(f_house_files, out_dir + "training.house.en-fr.fr")

e_senate_files = glob.glob("/people/vu/Projects/2017-NeuralAlignments/data/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/senate/debates/development/training/*.e")
f_senate_files = glob.glob("/people/vu/Projects/2017-NeuralAlignments/data/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/senate/debates/development/training/*.f")
write_file_list(e_senate_files, out_dir + "training.senate.en-fr.en")
write_file_list(f_senate_files, out_dir + "training.senate.en-fr.fr")
print("Done !")

Processing file: hansard.36.1.house.debates.001.e
Processing file: hansard.36.1.house.debates.002.e


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 182: invalid continuation byte